In [1]:
##################################
#
# Initial implementation of linear logic recurrent neural network
#
# The architecture is a modified RNN, see the paper "Linear logic and recurrent neural networks".
# Our inputs are sequences of symbols taken from an alphabet of size num_classes. The length
# of the sequences is N. Our outputs are also sequences of length N from the same alphabet.
#
# An input symbol is encoded as a one_hot vector, so our inputs are sequences of one-hot vectors
# of size num_classes. Put another way, this is the dimension of the input space.
#
# The function to be learned is func_to_learn
#
# At present at each time step the RNN simply emits its hidden state. These outputs are
# then passed through a fully-connected layer (one by one) to generate the output symbol
# at each time step.

# TODO:
#
# Dropout
# Layer normalisation (see "Recurrent neural networks in TensorFlow II")
# Hessian-free optimisation (see Sutskever paper)
#
# We currently only implement the Church numerals (i.e. Pi from the paper is empty)

# QUESTION:
#
# How to deal with output sequences of a different length to the input? Probably should feed
# the outputs at each time step into a second RNN

# NOTES
#
# We focus on experiments involving fairly local transformations of the sequence. Our
# aim is not to improve the ability of RNNs to capture long-range correlations (for this
# one should use memory). Instead, our aim is to enable higher complexity local transformations.

# GLOBAL FLAGS
num_classes = 2
batch_size = 1000
state_size = 35 # dimension of the state space H
operator_size = 5 # dimension of the auxiliary space K
input_size = num_classes # dimension of the input space I
N = 15 # length of sequences
training_percent = 0.05 # precentage used for training
epoch = 2
use_linearlogic = 0 # if 0, we compute an ordinary Elman RNN

# PROGRAM LIBRARY
Lambda = [1,2] # which Church numerals to couple into the RNN

In [2]:
total_arg_size = input_size + state_size
num_weights = total_arg_size*state_size + state_size # H,U,B

if use_linearlogic:
    num_weights += 2*operator_size*state_size # P,Q
    num_weights += 2*operator_size + input_size*operator_size # C,D,V
    num_weights += 2*operator_size*state_size # I,J
            
# This weight count does not include the weights in the output y, since they
# are in common between all our RNN models
print("Number of weights: " + str(num_weights))

Number of weights: 1330


In [3]:
# The next three lines are recommend by TF
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import numpy as np
import collections
import six
import math
import time

from random import shuffle
from tensorflow.python.ops import variable_scope as vs
from tensorflow.python.ops.rnn_cell import RNNCell
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import init_ops
from tensorflow.python.ops.math_ops import sigmoid
from tensorflow.python.ops.math_ops import tanh

In [4]:
def church(n,T):
    if n == 1:
        return(T)
    if n == 2:
        return(tf.mul(T,T))
    if n == 3:
        return(tf.mul(T,tf.mul(T,T)))
    
    # TODO higher powers
    return(T)

In [5]:
# In tf.nn.rnn_cell there is a private function _linear that 
# we have modified here to be used in our LinearLogicRNNCell

# A 2D tensor of shape [X,Y] means a matrix with X rows and Y cols
# The row index here is interpreted as indexing into a batch.
    
class LinearLogicRNNCell(RNNCell):
    
    def __init__(self, num_units, input_size=None, activation=tanh):
        if input_size is not None:
            logging.warn("%s: The input_size parameter is deprecated." % self)
        self._num_units = num_units
        self._activation = activation
    
    @property
    def state_size(self):
        return self._num_units

    @property
    def output_size(self):
        return self._num_units

    def __call__(self, inputs, state, scope=None):
        # the scope business gives a namespace to our weight variable matrix names
        with vs.variable_scope(scope or "Linear"): 
            # inputs has shape [batch_size, input_size]
            # state has shape [batch_size, state_size]
            args = [inputs,state]
            total_arg_size = input_size + state_size
    
            # NOTE: the H and U that are referred to in the RNN update equation of
            # the paper are the block parts of the following matrix HU.
            # Also, everything is transposed here compared to the paper, since we
            # need rows to correspond to entries in the batch in TensorFlow. So for
            # example we actually compute hH + xU + B.
            
            HU = vs.get_variable("HU", [total_arg_size, state_size])
            B = vs.get_variable("B", [state_size], initializer=init_ops.constant_initializer(0.0))
            
            # array_ops.concat(1,args) has shape [batch_size, input_size + state_size]
            # multiplying this with the block matrix HU = (H|U) computes hH + xU 
            res = tf.matmul(array_ops.concat(1, args), HU)
            
            if use_linearlogic and len(Lambda) > 0:
                # Lambda is defined globally, and is the list of Church numerals
                # The notation here matches Definition 2.9 of the paper
                I = vs.get_variable("I", [operator_size,state_size])
                J = vs.get_variable("J", [state_size,operator_size])
                V = vs.get_variable("V", [input_size,operator_size])
                
                Ps = [vs.get_variable("P" + str(n), [state_size,operator_size]) for n in Lambda]
                Cs = [vs.get_variable("C" + str(n), [operator_size]) for n in Lambda]
                ps = [self._activation(tf.matmul(state,P) + C) for P,C in zip(Ps,Cs)] # entry shape [batch_size,operator_size]                    
                make_diag = lambda A: tf.diag(A)
                ps_diag = [tf.map_fn(make_diag,p) for p in ps] # entry shape [batch_size,operator_size,operator_size]
                
                # Compute the Church numerals
                Vx = tf.matmul(inputs,V) # shape [batch_size,operator_size]
                church_outputs = [church(n,Vx) for n in Lambda]                
                Vx_diag = [tf.map_fn(make_diag,c) for c in church_outputs] # entry shape [batch_size,operator_size,operator_size]
                Jh = tf.matmul(state,J) # shape [batch_size,operator_size]
                Jh = tf.expand_dims(Jh,1) # shape [batch_size,1,operator_size]
                VxJh = [tf.batch_matmul(Jh,c) for c in Vx_diag] # entry shape [batch_size,1,operator_size]
                pVxJh = [tf.batch_matmul(c,p) for c,p in zip(VxJh,ps_diag)] # entry shape [batch_size,1,operator_size]
                pVxJh = [tf.reshape(c,[-1,operator_size]) for c in pVxJh] # entry shape [batch_size,operator_size]
                IpVxJh = [tf.matmul(c,I) for c in pVxJh] # entry shape [batch_size,state_size]
                church_sum = tf.add_n(IpVxJh) # shape [batch_size,state_size]
                
                output = self._activation(res + church_sum + B)
            else:
                output = self._activation(res + B)
                
        return output, output
        # note that as currently written the RNN emits its internal state at each time step

In [6]:
# The function from sequences to sequences that we will try to learn

def f_ident(seq):
    return seq

def f_reverse(seq):
    t = [0]*len(seq)
    for j in range(len(seq)):
        t[len(seq)-j-1] = seq[j]
    return t

def f_swap01(seq):
    t = []
    for j in range(len(seq)):
        if seq[j] == 0:
            t.append(0)
        else:
            t.append(1)
    return t

def f1(seq):
    t = []
    for j in range(len(seq)):
        if seq[j] == 1 and j > 0 and seq[j-1] == 1:
            if j > 3:
                t.append(seq[j-2])
            else:
                t.append(0)
        else:
            t.append(1)
    return t

def f_skiprepeat(seq):
    t = []
    for j in range(len(seq)):
        if j % 2 == 0:
            t.append(seq[j])
        else:
            t.append(seq[j-1])
    return t

def f_zeromeansrepeat(seq):
    t = []
    for j in range(len(seq)):
        if j > 0 and seq[j-1] == 0:
            t.append(1)
        else:
            t.append(seq[j])
    return t

func_to_learn = f_zeromeansrepeat

In [7]:
# Create a shuffled list of all binary sequences of length N
s = '{0:0' + str(N) + 'b}'
seq_input = [s.format(i) for i in range(2**N)]
shuffle(seq_input)
seq_input = [map(int,i) for i in seq_input]
ti = []
for i in seq_input:
    temp_list = []
    for j in i:
        temp_list.append(j)
    ti.append(temp_list)
seq_input = ti

print("Number of sequences: " + str(len(seq_input)))
print(seq_input[0])
# A typical element of seq_input at this point will be an array like
# array([[1],[0],[1],[1],[0]])

one_hots = []
for i in range(num_classes):
    a = [0.0]*num_classes
    a[i] = 1.0
    one_hots.append(np.array(a))

seq_input_onehot = []
for i in seq_input:
    temp_list = []
    for j in i:
        temp_list.append(one_hots[j])
    seq_input_onehot.append(np.array(temp_list))

print(seq_input_onehot[0])

Number of sequences: 32768
[1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0]
[[ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]]


In [8]:
# Training output
seq_output = []

# Swaps 0s for 1s in each sequence
for i in seq_input:
    seq_output.append(func_to_learn(i))
                    
print(seq_output[0])

seq_output_onehot = []
for i in seq_output:
    temp_list = []
    for j in i:
        temp_list.append(one_hots[j])
    seq_output_onehot.append(np.array(temp_list))

print(seq_output_onehot[0])

[1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0]
[[ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]]


In [9]:
NUM_EXAMPLES = int(training_percent * len(seq_input))
print("Number of training examples: " + str(NUM_EXAMPLES) + "/" + str(len(seq_input)))

test_input = seq_input_onehot[NUM_EXAMPLES:3*NUM_EXAMPLES]
test_output = seq_output_onehot[NUM_EXAMPLES:3*NUM_EXAMPLES]
train_input = seq_input_onehot[:NUM_EXAMPLES]
train_output = seq_output_onehot[:NUM_EXAMPLES]

print("")
print("The first one-hot encoded digit of the first three output sequences")
print(test_output[0][0])
print(test_output[1][0])
print(test_output[2][0])

Number of training examples: 1638/32768

The first one-hot encoded digit of the first three output sequences
[ 0.  1.]
[ 0.  1.]
[ 0.  1.]


In [10]:
# Definition of the model

# inputs, we create N of them, each of shape [None,input_size], one for
# each position in the sequence
inputs = [tf.placeholder(tf.float32, [None,input_size]) for _ in range(N)]
targets = [tf.placeholder(tf.float32, [None,input_size]) for _ in range(N)]

# We use tf.nn.rnn rather than dynamic_rnn because there appears to
# be a problem with tf.map_fn and the latter, at least in 0.10
# state_size is the number of hidden neurons in each layer
cell = LinearLogicRNNCell(state_size)

# tf.nn.rnn returns a pair, the first is a list of the
# outputs from each step, the second is the final internal state.
rnn_outputs, last_state = tf.nn.rnn(cell,inputs,dtype=tf.float32)

# Final fully connected layer
E = tf.Variable(tf.truncated_normal([state_size,input_size]))
F = tf.Variable(tf.constant(0.1, shape=[input_size]))

# prediction is a length N list of tensors of shape [None,input_size], where
# the jth row of prediction[d] is, for the jth input sequence in the batch,
# the probability distribution over symbols for the output symbol in position d.
logits = [tf.matmul(rnn_output, E) + F for rnn_output in rnn_outputs]
prediction = [tf.nn.softmax(logit) for logit in logits] 
ce = [tf.reduce_sum(targets[i] * tf.log(prediction[i])) for i in range(N)]

cross_entropy = -tf.add_n(ce)
optimizer = tf.train.AdamOptimizer()
minimize = optimizer.minimize(cross_entropy)

mistakes = [tf.not_equal(tf.argmax(targets[i], 1), tf.argmax(prediction[i], 1)) for i in range(N)]
errors = [tf.reduce_mean(tf.cast(m, tf.float32)) for m in mistakes]

In [11]:
# Initialise the model
init_op = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init_op)

In [12]:
# Display the errors before training
feed_dict = {}
for d in range(N):
    in_node = inputs[d]
    out_node = targets[d]
    
    ti = []
    to = []
    for k in range(len(test_input)):
        ti.append(test_input[k][d]) # A vector giving the one-hot encoding of the dth symbol in the kth sequence
        to.append(test_output[k][d])
    feed_dict[in_node] = np.array(ti)
    feed_dict[out_node] = np.array(to)

# The first three digits of this should match the printout for the
# first three test output sequences given earlier
print(sess.run(tf.argmax(targets[0],1),feed_dict))
print(sess.run(tf.argmax(prediction[0],1),feed_dict))
print(sess.run(tf.not_equal(tf.argmax(targets[0], 1), tf.argmax(prediction[0], 1)),feed_dict))

print("")
print("The mean of the errors in each digit for the test set:")
incorrects = sess.run(errors, feed_dict)
print(incorrects)
print("Mean: " + str(np.mean(incorrects)))

[1 1 1 ..., 0 1 1]
[0 0 0 ..., 1 0 0]
[ True  True  True ...,  True  True  True]

The mean of the errors in each digit for the test set:
[1.0, 0.25518927, 0.24725275, 0.50091577, 0.45665446, 0.38583639, 0.3907204, 0.52014655, 0.46214896, 0.47466421, 0.4661172, 0.47893772, 0.48412699, 0.47771671, 0.47466421]
Mean: 0.471673


In [13]:
pre_train_time = time.time()

# Training
no_of_batches = int(len(train_input)/batch_size)
print("Number of batches: " + str(no_of_batches))

# An annoying thing here is that we cannot use a list as a key in a 
# dictionary. The workaround we found on StackOverflow here:
# http://stackoverflow.com/questions/33684657/issue-feeding-a-list-into-feed-dict-in-tensorflow)

# epoch is a global var
for i in range(epoch):
    ptr = 0
    for j in range(no_of_batches):
        inp = train_input[ptr:ptr+batch_size]
        out = train_output[ptr:ptr+batch_size]
        ptr += batch_size
        
        feed_dict = {}
        for d in range(N):
            in_node = inputs[d]
            out_node = targets[d]
            
            # inp has dimensions [batch_size, N, num_classes] and we want to extract
            # the 2D Tensor of shape [batch_size, num_classes] obtained by setting the
            # second coordinate to d
            ti = []
            to = []
            for k in range(batch_size):
                ti.append(inp[k][d])
                to.append(out[k][d])

            feed_dict[in_node] = np.array(ti)
            feed_dict[out_node] = np.array(to)
            
        sess.run(minimize, feed_dict)
    print("Epoch - " + str(i+1))
    
print("It took", time.time() - pre_train_time, "seconds to train.")

Number of batches: 1
Epoch - 1
Epoch - 2
It took 0.243247032166 seconds to train.


In [14]:
# Display the errors after training
feed_dict = {}
for d in range(N):
    in_node = inputs[d]
    out_node = targets[d]
    
    ti = []
    to = []
    for k in range(len(test_input)):
        ti.append(test_input[k][d]) # A vector giving the one-hot encoding of the dth symbol in the kth sequence
        to.append(test_output[k][d])
    feed_dict[in_node] = np.array(ti)
    feed_dict[out_node] = np.array(to)

# The first three digits of this should match the printout for the
# first three test output sequences given earlier
data = sess.run([tf.argmax(targets[0],1),
                 tf.argmax(prediction[0],1),
                 cross_entropy],feed_dict)

print("First digits of test outputs (actual)")
print(data[0])
print("First digits of test outputs (predicted)")
print(data[1])
print("Cross-entropy: " + str(data[2]))

# print the mean of the errors in each digit for the test set.
incorrects = sess.run(errors, feed_dict)
print(incorrects)
print("Mean: " + str(np.mean(incorrects)))
sess.close()

First digits of test outputs (actual)
[1 1 1 ..., 0 1 1]
First digits of test outputs (predicted)
[0 0 0 ..., 1 0 0]
Cross-entropy: 42690.9
[1.0, 0.25518927, 0.24725275, 0.30525032, 0.34157509, 0.23962149, 0.32631257, 0.41758242, 0.36416361, 0.36691087, 0.37210011, 0.38766789, 0.38705739, 0.38644689, 0.40018314]
Mean: 0.386488


In [15]:
# N = 15, church = 1, 198 seconds to train
# N = 15, church = 2, 205 seconds

# for skiprepeat N = 15 without linearlogic, 
# [0.0, 0.2530525, 0.3768315, 0.18315019, 0.24832112, 0.25457877, 0.19948107, 0.25457877, 0.25198412, 0.25839439, 0.2269536, 0.26862028, 0.23076923, 0.26678878, 0.22710623]
# with linear logic (church = 2), all zeros, training time 205

# for skiprepeat N = 20 without linearlogic,

# N = 15, training_percent = 0.1, epoch = 10, use_linearlogic = 1, Lambda = [1,2]
# 1187 seconds to train
# [0.0, 0.0, 0.498779, 0.061660562, 0.37057388, 0.12713675, 0.33653846, 0.17231379, 0.30525032, 0.17185593, 0.31913918, 0.17918193, 0.32341269, 0.1802503, 0.32066545]
# mean 0.2244

# N = 15, training_percent = 0.1, epoch = 10, use_linearlogic = 0, Lambda = [1,2]
# 1 second to train
# mean 0.233384

In [16]:
# 1220 weights hidden = 25 (epoch = 2, mean = 0.315608)
# 1330 weights hidden = 35 (epoch = 2, (no LL) mean = 0.386488)